In [1]:
#Import TSPICE and initialize the kernels
import tspice as tsp
tsp.initialize()

/home/deivyastro/TSPICE_package/src/tspice/tidal_signal.py:4: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 2.3.5)
  import scipy.special as sps


TSPICE initialized successfully. Kernels loaded from: /home/deivyastro/TSPICE_package/src/tspice/data/meta_kernel


In [17]:
import numpy as np

In [2]:
#Uploading the Prem model with modifications suggested by from Amorin & Gudkova (2025)
from PREM.data import prem_amorim as prem

#Define the planet profiles
planet_profile = {'rho': prem.rho_r_interp,
				'lamb': prem.lamb_r_interp,
				'mu': prem.mu_r_interp,
				'g': prem.g_r_interp,
				'dimensionless': False}

In [3]:
#Create Earth interior model
earth_interior = tsp.BodyResponse('Earth')

#Define layers for our planetary model
earth_interior.scale_constants(verbose=True)
layers_list = [dict(name='Outer Core', type='fluid', r0=1221500.0, rf=3480000.0),
               dict(name='Inner Core', type='solid', r0=0, rf=1221500.0),
               dict(name='Mantle + crust', type='solid', r0=3480000.0, rf = earth_interior.L)]


Characteristic scales for adimensionalization:
Length scale L = 6.37e+06 m
Mass scale M = 5.97e+24 kg
Density scale RHO = 2.31e+04 kg/m^3
Pressure/Elasticity scale P = 1.44e+12 Pa
Velocity scale V = 7.91e+03 m/s
Time scale T = 8.05e+02 s
Angular frequency scale OMEGA = 1.24e-03 rad/s
Gravity scale Gad = 9.82e+00 m/s^2


## Comparison with Amorin & Gudkova (2024):

In [7]:
#Set parameters for the integration	
n = 2
f_day = 1.9323  #M2 tide in cycles/day
r0_ini = 6e3
earth_interior.set_integration_parameters_ad(n=n, f_days=f_day, layers_list=layers_list, planet_profile=planet_profile, r0_ini=r0_ini)

#Integration
earth_interior.integrate_internal_solutions_ad(verbose=False)

#Get Love numbers
h_2_tsp, l_2_tsp, k_2_tsp = earth_interior.h_n, earth_interior.l_n, earth_interior.k_n
print(f"Computed Love numbers for n={n} at f={f_day} cpd:")
print(f"h_{n} = {h_2_tsp}")
print(f"l_{n} = {l_2_tsp}")
print(f"k_{n} = {k_2_tsp}")

Computed Love numbers for n=2 at f=1.9323 cpd:
h_2 = 0.6093101339403031
l_2 = 0.08563952001113084
k_2 = 0.2990408649998255


In [8]:
#Set parameters for the integration	
n = 3
f_day = 1.9323  #M2 tide in cycles/day
r0_ini = 6e3
earth_interior.set_integration_parameters_ad(n=n, f_days=f_day, layers_list=layers_list, planet_profile=planet_profile, r0_ini=r0_ini)

#Integration
earth_interior.integrate_internal_solutions_ad(verbose=False)

#Get Love numbers
h_3_tsp, l_3_tsp, k_3_tsp = earth_interior.h_n, earth_interior.l_n, earth_interior.k_n
print(f"Computed Love numbers for n={n} at f={f_day} cpd:")
print(f"h_{n} = {h_3_tsp}")
print(f"l_{n} = {l_3_tsp}")
print(f"k_{n} = {k_3_tsp}")

Computed Love numbers for n=3 at f=1.9323 cpd:
h_3 = 0.30487694085970873
l_3 = 0.03090066361224708
k_3 = 0.09285105820385264


In [9]:
#Amorim & Gudkova (2025) results
h_2_ag = 0.60496
l_2_ag = 0.08399
k_2_ag = 0.29872
k_3_ag = 0.09203

In [15]:
#Difference between TSPICE and Amorim & Gudkova (2025)
print("\nDifferences with Amorim & Gudkova (2025) results:")
print(f"Delta h_2 = {h_2_tsp - h_2_ag:.4f}")
print(f"Delta l_2 = {l_2_tsp - l_2_ag:.4f}")
print(f"Delta k_2 = {k_2_tsp - k_2_ag:.4f}")
print(f"Delta k_3 = {k_3_tsp - k_3_ag:.4f}")


#Percentage differences
print("\nPercentage differences with Amorim & Gudkova (2025) results:")
print(f"Percentage Delta h_2 = {(h_2_tsp - h_2_ag)/h_2_ag * 100:.3f} %")
print(f"Percentage Delta l_2 = {(l_2_tsp - l_2_ag)/l_2_ag * 100:.3f} %")
print(f"Percentage Delta k_2 = {(k_2_tsp - k_2_ag)/k_2_ag * 100:.3f} %")
print(f"Percentage Delta k_3 = {(k_3_tsp - k_3_ag)/k_3_ag * 100:.3f} %")


Differences with Amorim & Gudkova (2025) results:
Delta h_2 = 0.0044
Delta l_2 = 0.0016
Delta k_2 = 0.0003
Delta k_3 = 0.0008

Percentage differences with Amorim & Gudkova (2025) results:
Percentage Delta h_2 = 0.719 %
Percentage Delta l_2 = 1.964 %
Percentage Delta k_2 = 0.107 %
Percentage Delta k_3 = 0.892 %


## Comparison with Xu & Sun (2003)

In [21]:
love_numbers_xu = np.array([[0.603306, 0.298256, 0.083949],
          [0.287109, 0.091773, 0.014849],
          [0.174801, 0.041418, 0.010151],
          [0.605630, 0.299365, 0.084042],
          [0.288168, 0.092162, 0.014698],
          [0.174977, 0.041437, 0.010147],
		[0.288848, 0.092362, 0.014644],
        [0.175158, 0.041479, 0.010134],
        [0.176197, 0.041700, 0.010060]])
love_numbers_xu

array([[0.603306, 0.298256, 0.083949],
       [0.287109, 0.091773, 0.014849],
       [0.174801, 0.041418, 0.010151],
       [0.60563 , 0.299365, 0.084042],
       [0.288168, 0.092162, 0.014698],
       [0.174977, 0.041437, 0.010147],
       [0.288848, 0.092362, 0.014644],
       [0.175158, 0.041479, 0.010134],
       [0.176197, 0.0417  , 0.01006 ]])

In [22]:
#To store the results and comparison
love_numbers_tsp = np.zeros_like(love_numbers_xu)
deltas = np.zeros_like(love_numbers_xu)
deltas_percent = np.zeros_like(love_numbers_xu)
love_numbers_tsp

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [29]:
#Frequencies to compare
fO1 = 0.92953571
fM2 = 1.9323
fM3 = 2.8984
fM4 = 3.8645
fs = [fO1, fO1, fO1, fM2, fM2, fM2, fM3, fM3, fM4]

#List of degrees
ns = [2,3,4,2,3,4,3,4,4]

#Cycles per day
for i,(f,n) in enumerate(zip(fs,ns)):
	
	#Set parameters for the integration
	earth_interior.set_integration_parameters_ad(n=n, f_days=f, layers_list=layers_list, planet_profile=planet_profile, r0_ini=r0_ini)

	#Integration
	earth_interior.integrate_internal_solutions_ad(verbose=False)

	#Get Love numbers
	h_n_tsp, l_n_tsp, k_n_tsp = earth_interior.h_n, earth_interior.l_n, earth_interior.k_n
	love_numbers_tsp[i,:] = [h_n_tsp, k_n_tsp, l_n_tsp]
	deltas[i,:] = love_numbers_tsp[i,:] - love_numbers_xu[i,:]
	deltas_percent[i,:] = deltas[i,:] / love_numbers_xu[i,:] * 100


	#print(f"Computed Love numbers for n={n} at f={f} cpd:")
	#print(f"h_{n} = {h_n_tsp}")
	#print(f"l_{n} = {l_n_tsp}")
	#print(f"k_{n} = {k_n_tsp}")

	#

In [27]:
for i,(f,n) in enumerate(zip(fs,ns)):
    print(f"Frequency: {f} cpd, Degree: {n}")

Frequency: 0.92953571 cpd, Degree: 2
Frequency: 0.92953571 cpd, Degree: 3
Frequency: 0.92953571 cpd, Degree: 4
Frequency: 1.93502 cpd, Degree: 2
Frequency: 1.93502 cpd, Degree: 3
Frequency: 1.93502 cpd, Degree: 4
Frequency: 2.8984 cpd, Degree: 3
Frequency: 2.8984 cpd, Degree: 4
Frequency: 3.8645 cpd, Degree: 4


In [30]:
love_numbers_tsp

array([[ 0.60775239,  0.29836193,  0.08461225],
       [ 0.29538563,  0.09415259,  0.02033032],
       [ 0.15155398,  0.03941253, -0.01667561],
       [ 0.60931013,  0.29904086,  0.08563952],
       [ 0.30487694,  0.09285106,  0.03090066],
       [ 0.15209321,  0.0385029 , -0.00549865],
       [ 0.2908257 ,  0.09465561, -0.00370402],
       [ 0.17222887,  0.04168116,  0.00398706],
       [ 0.16391725,  0.03974359,  0.00070912]])

In [31]:
deltas

array([[ 0.00444639,  0.00010593,  0.00066325],
       [ 0.00827663,  0.00237959,  0.00548132],
       [-0.02324702, -0.00200547, -0.02682661],
       [ 0.00368013, -0.00032414,  0.00159752],
       [ 0.01670894,  0.00068906,  0.01620266],
       [-0.02288379, -0.0029341 , -0.01564565],
       [ 0.0019777 ,  0.00229361, -0.01834802],
       [-0.00292913,  0.00020216, -0.00614694],
       [-0.01227975, -0.00195641, -0.00935088]])

In [32]:
deltas_percent

array([[ 7.37003685e-01,  3.55150762e-02,  7.90066645e-01],
       [ 2.88274895e+00,  2.59291089e+00,  3.69137424e+01],
       [-1.32991333e+01, -4.84202383e+00, -2.64275531e+02],
       [ 6.07653838e-01, -1.08274180e-01,  1.90085911e+00],
       [ 5.79833322e+00,  7.47659777e-01,  1.10237200e+02],
       [-1.30781678e+01, -7.08087388e+00, -1.54189901e+02],
       [ 6.84683968e-01,  2.48328724e+00, -1.25293778e+02],
       [-1.67227731e+00,  4.87388541e-01, -6.06565865e+01],
       [-6.96932769e+00, -4.69162003e+00, -9.29510950e+01]])

In [47]:
100*(love_numbers_tsp[3,:] - love_numbers_xu[3,:])/love_numbers_xu[3,:]

array([ 0.60765384, -0.10827418,  1.90085911])

In [48]:
100*np.abs(love_numbers_tsp[3,:] - love_numbers_xu[3,:])/love_numbers_xu[3,:]

array([0.60765384, 0.10827418, 1.90085911])

In [43]:
love_numbers_xu[3,:]

array([0.60563 , 0.299365, 0.084042])

In [49]:
100*np.abs(love_numbers_tsp[4,:] - love_numbers_xu[4,:])/love_numbers_xu[4,:]

array([  5.79833322,   0.74765978, 110.2371997 ])